In [11]:
import cv2
import numpy as np
import copy
import math
import os

def calculateFingers(res, drawing):
    #  convexity defect
    hull = cv2.convexHull(res, returnPoints=False)
    if len(hull) > 3:
        defects = cv2.convexityDefects(res, hull)
        if defects is not None:
            cnt = 0
            for i in range(defects.shape[0]):  # calculate the angle
                s, e, f, d = defects[i][0]
                start = tuple(res[s][0])
                end = tuple(res[e][0])
                far = tuple(res[f][0])
                a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # cosine theorem
                if angle <= math.pi / 2:  # angle less than 90 degree, treat as fingers
                    cnt += 1
                    cv2.circle(drawing, far, 8, [211, 84, 0], -1)
            if cnt > 0:
                return True, cnt+1
            else:
                return True, 0
    return False, 0

# Open Camera
camera = cv2.VideoCapture(0)
camera.set(10, 200)#while True:
while camera.isOpened():
    #Main Camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    ret, frame = camera.read()
    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # Smoothing
    frame = cv2.flip(frame, 1)  #Horizontal Flip
    cv2.imshow('original', frame)
       #Background Removal
    bgModel = cv2.createBackgroundSubtractorMOG2(0, 50)
    fgmask = bgModel.apply(frame)
    kernel = np.ones((3, 3), np.uint8)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    img = cv2.bitwise_and(frame, frame, mask=fgmask)
    cv2.imshow("temp", img)

    
    # Skin detect and thresholding
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 48, 80], dtype="uint8")
    upper = np.array([20, 255, 255], dtype="uint8")
    skinMask = cv2.inRange(hsv, lower, upper)
    cv2.imshow('Threshold Hands', skinMask)     # Getting the contours and convex hull
    skinMask1 = copy.deepcopy(skinMask)
    contours, hierarchy = cv2.findContours(skinMask1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    length = len(contours)
    maxArea = -1
    if length > 0:
        for i in range(length):
            temp = contours[i]
            area = cv2.contourArea(temp)
            if area > maxArea:
                maxArea = area
                ci = i
                res = contours[ci]
        hull = cv2.convexHull(res)
        drawing = np.zeros(img.shape, np.uint8)
        cv2.drawContours(drawing, [res], 0, (0, 255, 0), 2)
        cv2.drawContours(drawing, [hull], 0, (0, 0, 255), 3)
        
        try:
            isFinishCal, cnt = calculateFingers(res, drawing)
        except:
            continue
        print("Fingers", cnt)
        cv2.imshow('output', drawing)
        k = cv2.waitKey(10)
        if k == 27:  # press ESC to exit
            break
            
camera.release()
cv2.destroyAllWindows()

1.0
31.5
1270.5
Fingers 2
6.0
8.5
0.0
5.0
11.5
149.5
14.5
130.0
0.0
16.0
8.0
200.0
111.0
15.0
0.5
5.0
610.0
Fingers 0
0.0
5.5
3.5
3.5
15.5
2.5
2.0
14.5
0.0
0.0
4.5
52.5
9.5
433.0
53.5
65.0
3.0
603.0
Fingers 3
0.0
5.5
3.5
3.5
15.5
2.5
2.0
14.5
0.0
0.0
4.5
52.5
9.5
433.0
53.5
65.0
3.0
603.0
Fingers 3
117.0
33.5
0.0
38.0
0.5
0.5
17.0
14.0
8.0
0.5
4.0
1.0
82.0
2.0
11.5
0.0
6.5
1.0
38.0
2.0
2.0
0.5
15.0
272.0
44.0
399.5
34.0
Fingers 0
0.0
0.0
3.0
0.5
0.0
1.0
13.0
58.5
0.0
24.5
2.0
0.0
10.0
7.5
3.5
8.0
14.5
46.5
1.0
6.0
81.0
0.0
43.0
5.5
0.0
291.0
201.5
0.5
251.0
0.0
Fingers 0
0.0
0.0
11.5
16.5
32.5
3.0
39.5
244.5
0.5
155.0
0.0
26.0
0.0
0.0
0.0
9.0
0.0
13.5
18.5
0.0
130.0
527.0
0.0
2.0
0.0
529.0
36.0
586.5
Fingers 2
12.0
1.0
9.0
0.0
15.0
5.5
0.0
11.5
0.0
82.0
19.5
7.0
8.5
0.0
3.0
0.0
0.0
78.5
365.0
3.5
218.5
369.0
Fingers 0
12.0
1.0
9.0
0.0
15.0
5.5
0.0
11.5
0.0
82.0
19.5
7.0
8.5
0.0
3.0
0.0
0.0
78.5
365.0
3.5
218.5
369.0
Fingers 0
20.0
7.5
32.0
24.5
32.0
9.0
0.0
2.5
103.0
46.5
0.0
0.0
0.0
2